In [12]:
import pandas as pd
import numpy as np

In [29]:
df_coal = pd.read_excel('./coal_month_ahead.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)
df_oil = pd.read_excel('./oil_month_ahead.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)
df_eua = pd.read_excel('./eua.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)
df_gas = pd.read_excel('./fuel_history.xlsx', sheet_name='Sheet1', index_col=0)
df_currency = pd.read_excel('./currency.xlsx', sheet_name='Part 0', skiprows=[0, 1, 2], index_col=0)

In [30]:
start_date = np.datetime64("2022-08-31") #pd.DateTime('20220901', tz='Europe/Amsterdam')
end_date = np.datetime64("2023-02-28") #pd.DateTime('20230228', tz='Europe/Amsterdam')

df_coal.index = pd.to_datetime(df_coal.index)
df_oil.index = pd.to_datetime(df_oil.index)
df_eua.index = pd.to_datetime(df_eua.index)
df_gas.index = pd.to_datetime(df_gas.index)
df_currency.index = pd.to_datetime(df_currency.index)

df_coal = df_coal.loc[(df_coal.index > start_date) & (df_coal.index <= end_date)]
df_oil = df_oil.loc[(df_oil.index > start_date) & (df_oil.index <= end_date)]
df_eua = df_eua.loc[(df_eua.index > start_date) & (df_eua.index <= end_date)]
df_gas = df_gas.loc[(df_gas.index > start_date) & (df_gas.index <= end_date)]
df_currency = df_currency.loc[(df_currency.index > start_date) & (df_currency.index <= end_date)]

display(df_coal)
display(df_oil)
display(df_eua)
display(df_gas)
display(df_currency)

,Front Year,Front Month
Date,,
2022-09-01,312.0,358.0
2022-09-02,323.0,367.0
2022-09-03,NaN,NaN
2022-09-04,NaN,NaN
2022-09-05,344.0,394.0
...,...,...
2023-02-24,161.0,156.0
2023-02-25,NaN,NaN
2023-02-26,NaN,NaN


,USD 12 Months,USD
Date,,
2022-09-01,81.04,92.36
2022-09-02,81.04,93.02
2022-09-03,NaN,NaN
2022-09-04,NaN,NaN
2022-09-05,81.67,95.74
...,...,...
2023-02-24,78.63,83.16
2023-02-25,NaN,NaN
2023-02-26,NaN,NaN


,Dec 23,Dec 22,Dec 21,Dec 24
Date,,,,
2022-09-01,84.13,81.04,NaN,89.05
2022-09-02,80.98,77.91,NaN,85.90
2022-09-03,80.98,77.91,NaN,85.90
2022-09-04,80.98,77.91,NaN,85.90
2022-09-05,77.02,73.94,NaN,82.08
...,...,...,...,...
2023-02-24,97.85,NaN,NaN,102.61
2023-02-25,97.85,NaN,NaN,102.61
2023-02-26,97.85,NaN,NaN,102.61


,Week,EPEX NL Base,EPEX NL Peak,CO2,TTF DA,API2,EUR/USD
Date,,,,,,,
2022-09-01,35,534.68,539.75,81.50,268.0,360.00,1.0013
2022-09-02,35,425.28,427.77,78.50,230.0,355.00,1.0053
2022-09-03,35,286.35,NaN,81.00,205.0,366.25,0.9944
2022-09-04,35,250.69,NaN,78.00,200.0,366.25,0.9944
2022-09-05,36,359.16,376.01,78.00,180.0,366.25,0.9944
...,...,...,...,...,...,...,...
2023-01-22,3,151.28,NaN,85.00,68.0,179.00,1.0831
2023-01-23,4,201.90,235.98,85.00,68.0,179.00,1.0831
2023-01-24,4,199.79,232.41,85.25,68.0,177.00,1.0858


,EURO to USD
Date,
2022-09-01,1.00
2022-09-02,1.00
2022-09-03,1.00
2022-09-04,1.00
2022-09-05,0.99
...,...
2023-02-24,1.06
2023-02-25,1.06
2023-02-26,1.06


In [34]:
df_fuel = pd.DataFrame()
df_fuel.index = df_coal.index
df_fuel['coal'] = df_coal['Front Month']
df_fuel['oil'] = df_oil['USD'] / df_currency['EURO to USD']
df_fuel['gas'] = df_gas['TTF DA']
df_fuel['eua'] = df_eua['Dec 24']

for index, row in df_fuel.iterrows():
    if np.isnan(row['coal']):
        loc = df_fuel.index.get_loc(index)
        df_fuel.loc[index, 'coal'] = df_fuel.iloc[loc - 1]['coal']
        
    if np.isnan(row['oil']):
        loc = df_fuel.index.get_loc(index)
        df_fuel.loc[index, 'oil'] = df_fuel.iloc[loc - 1]['oil']

display(df_fuel)
df_fuel.to_excel("./for_io/fuel_prices.xlsx")

,coal,oil,gas,eua
Date,,,,
2022-09-01,358.0,92.360000,268.0,89.05
2022-09-02,367.0,93.020000,230.0,85.90
2022-09-03,367.0,93.020000,205.0,85.90
2022-09-04,367.0,93.020000,200.0,85.90
2022-09-05,394.0,96.707071,180.0,82.08
...,...,...,...,...
2023-02-24,156.0,78.452830,NaN,102.61
2023-02-25,156.0,78.452830,NaN,102.61
2023-02-26,156.0,78.452830,NaN,102.61
